# Python-Specific Interview Problems

This notebook contains problems focused on advanced Python features and idioms commonly tested in Python developer interviews.

## Table of Contents
1. [Decorators](#decorators)
2. [Context Managers](#context-managers)
3. [Generators & Iterators](#generators)
4. [Metaclasses](#metaclasses)
5. [Magic Methods](#magic-methods)
6. [Python Internals](#internals)

<a id='decorators'></a>
## 1. Decorators

### Problem 1: Implement a Retry Decorator (Medium)

**Problem Statement:**
Create a decorator that automatically retries a function if it raises an exception. The decorator should:
- Accept `max_retries` and `delay` parameters
- Wait between retries
- Re-raise the exception if all retries fail

**Example Usage:**
```python
@retry(max_retries=3, delay=1)
def flaky_api_call():
    # May fail sometimes
    pass
```

In [ ]:
import time
import functools
from typing import Callable, Type

def retry(max_retries: int = 3, delay: float = 1, 
          exceptions: tuple = (Exception,)):
    """
    Decorator that retries a function on failure.
    
    Args:
        max_retries: Maximum number of retry attempts
        delay: Seconds to wait between retries
        exceptions: Tuple of exceptions to catch
    """
    def decorator(func: Callable) -> Callable:
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(max_retries + 1):
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    last_exception = e
                    if attempt < max_retries:
                        print(f"Attempt {attempt + 1} failed. Retrying in {delay}s...")
                        time.sleep(delay)
                    else:
                        print(f"All {max_retries} retries failed.")
            
            # Re-raise the last exception
            raise last_exception
        
        return wrapper
    return decorator

# Test the decorator
counter = 0

@retry(max_retries=3, delay=0.1)
def flaky_function():
    global counter
    counter += 1
    if counter < 3:
        raise ValueError(f"Failed on attempt {counter}")
    return "Success!"

result = flaky_function()
assert result == "Success!"
assert counter == 3
print("✓ Retry decorator test passed!")

**Key Concepts:**
- Decorator with parameters requires nested functions (3 levels)
- Use `functools.wraps` to preserve function metadata
- Handle exceptions properly
- Support both `*args` and `**kwargs`

**Follow-up Questions:**
- How would you add exponential backoff?
- How to make it work with async functions?
- How to log retry attempts?

### Problem 2: Implement a Memoization Decorator (Medium)

**Problem Statement:**
Create a decorator that caches function results based on arguments. Should handle:
- Both positional and keyword arguments
- Cache size limit
- Cache statistics (hits/misses)

**Example:**
```python
@memoize(max_size=100)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)
```

In [ ]:
import functools
from collections import OrderedDict
from typing import Callable, Any

def memoize(max_size: int = 128):
    """
    Decorator that caches function results with LRU eviction.
    
    Args:
        max_size: Maximum number of cached results
    """
    def decorator(func: Callable) -> Callable:
        cache = OrderedDict()
        stats = {'hits': 0, 'misses': 0}
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # Create hashable key from args and kwargs
            key = (args, tuple(sorted(kwargs.items())))
            
            if key in cache:
                # Cache hit - move to end (most recently used)
                stats['hits'] += 1
                cache.move_to_end(key)
                return cache[key]
            
            # Cache miss - compute result
            stats['misses'] += 1
            result = func(*args, **kwargs)
            
            # Add to cache
            cache[key] = result
            
            # Evict oldest if over size limit
            if len(cache) > max_size:
                cache.popitem(last=False)  # Remove oldest (FIFO)
            
            return result
        
        # Add cache inspection methods
        def cache_info():
            return {
                'hits': stats['hits'],
                'misses': stats['misses'],
                'size': len(cache),
                'max_size': max_size
            }
        
        def cache_clear():
            cache.clear()
            stats['hits'] = stats['misses'] = 0
        
        wrapper.cache_info = cache_info
        wrapper.cache_clear = cache_clear
        
        return wrapper
    return decorator

# Test the memoization decorator
@memoize(max_size=10)
def fibonacci(n: int) -> int:
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

# First call - cache misses
result = fibonacci(10)
assert result == 55

# Second call - cache hits
fibonacci.cache_clear()
fibonacci(5)
fibonacci(5)  # Should hit cache
info = fibonacci.cache_info()
assert info['hits'] > 0

print(f"✓ Memoization decorator test passed! Cache stats: {info}")

**Advanced Decorator Concepts:**
- Creating hashable keys from arguments
- Using `OrderedDict` for LRU behavior
- Adding methods to wrapped function
- Closure variables for state management

**Comparison with `functools.lru_cache`:**
```python
from functools import lru_cache

@lru_cache(maxsize=128)
def fibonacci(n):
    # Built-in version
    pass
```

**Follow-up Questions:**
- How to handle unhashable arguments (lists, dicts)?
- How to add TTL (time-to-live) for cache entries?
- Thread safety considerations?

### Problem 3: Rate Limiter Decorator (Medium)

**Problem Statement:**
Create a decorator that limits how often a function can be called.

**Requirements:**
- Allow max `n` calls per `time_window` seconds
- Block or raise exception when limit exceeded
- Track calls per function, not globally

In [ ]:
import time
import functools
from collections import deque
from typing import Callable

class RateLimitExceeded(Exception):
    """Raised when rate limit is exceeded"""
    pass

def rate_limit(max_calls: int, time_window: float):
    """
    Decorator that limits function calls to max_calls per time_window seconds.
    
    Args:
        max_calls: Maximum number of calls allowed
        time_window: Time window in seconds
    """
    def decorator(func: Callable) -> Callable:
        # Store timestamps of recent calls
        calls = deque()
        
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            now = time.time()
            
            # Remove calls outside the time window
            while calls and calls[0] < now - time_window:
                calls.popleft()
            
            # Check if we've exceeded the limit
            if len(calls) >= max_calls:
                # Calculate when the next call can be made
                wait_time = time_window - (now - calls[0])
                raise RateLimitExceeded(
                    f"Rate limit exceeded. Try again in {wait_time:.2f} seconds."
                )
            
            # Record this call
            calls.append(now)
            
            return func(*args, **kwargs)
        
        return wrapper
    return decorator

# Test the rate limiter
@rate_limit(max_calls=3, time_window=1.0)
def api_call():
    return "API response"

# First 3 calls should succeed
for i in range(3):
    assert api_call() == "API response"

# 4th call should fail
try:
    api_call()
    assert False, "Should have raised RateLimitExceeded"
except RateLimitExceeded as e:
    print(f"✓ Rate limiter working: {e}")

# After waiting, should work again
time.sleep(1.1)
assert api_call() == "API response"
print("✓ All rate limiter tests passed!")

**Rate Limiting Strategies:**
1. **Token Bucket** - Refill tokens at fixed rate
2. **Leaky Bucket** - Process requests at constant rate
3. **Sliding Window** - Track calls in time window (implemented above)
4. **Fixed Window** - Reset counter at fixed intervals

**Follow-up Questions:**
- How to implement per-user rate limiting?
- How to make it distributed across multiple servers?
- What about async functions?

<a id='context-managers'></a>
## 2. Context Managers

### Problem 4: Database Transaction Context Manager (Medium)

**Problem Statement:**
Implement a context manager for database transactions that:
- Automatically commits on success
- Rolls back on exception
- Properly closes connection
- Can be used as decorator or context manager

**Example Usage:**
```python
with transaction(db_connection) as cursor:
    cursor.execute("INSERT ...")
    # Auto-commit if no exception
    # Auto-rollback if exception
```

In [ ]:
from contextlib import contextmanager
from typing import Any

# Mock database connection for demonstration
class MockConnection:
    def __init__(self):
        self.committed = False
        self.rolled_back = False
        self.closed = False
    
    def cursor(self):
        return self
    
    def execute(self, query: str):
        if "error" in query.lower():
            raise ValueError("Query error")
        return "Result"
    
    def commit(self):
        self.committed = True
    
    def rollback(self):
        self.rolled_back = True
    
    def close(self):
        self.closed = True

# Implementation 1: Class-based context manager
class Transaction:
    """
    Context manager for database transactions.
    Implements __enter__ and __exit__ methods.
    """
    
    def __init__(self, connection):
        self.connection = connection
        self.cursor = None
    
    def __enter__(self):
        """Called when entering 'with' block"""
        self.cursor = self.connection.cursor()
        return self.cursor
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """
        Called when exiting 'with' block.
        
        Args:
            exc_type: Exception type (None if no exception)
            exc_val: Exception value
            exc_tb: Exception traceback
        
        Returns:
            False to re-raise exception, True to suppress
        """
        try:
            if exc_type is None:
                # No exception - commit
                self.connection.commit()
            else:
                # Exception occurred - rollback
                self.connection.rollback()
        finally:
            # Always close cursor
            if self.cursor:
                self.cursor.close()
        
        # Return False to re-raise exception
        return False

# Implementation 2: Generator-based context manager
@contextmanager
def transaction(connection):
    """
    Context manager using @contextmanager decorator.
    Uses generator with yield.
    """
    cursor = connection.cursor()
    try:
        # Code before yield runs on __enter__
        yield cursor
        # If we get here, no exception - commit
        connection.commit()
    except Exception:
        # Exception occurred - rollback
        connection.rollback()
        raise  # Re-raise the exception
    finally:
        # Code in finally always runs
        cursor.close()

# Test class-based context manager
conn = MockConnection()
with Transaction(conn) as cursor:
    cursor.execute("SELECT * FROM users")
assert conn.committed == True
assert conn.rolled_back == False
print("✓ Transaction committed successfully")

# Test rollback on exception
conn = MockConnection()
try:
    with transaction(conn) as cursor:
        cursor.execute("INSERT error")
except ValueError:
    pass
assert conn.committed == False
assert conn.rolled_back == True
print("✓ Transaction rolled back on error")
print("✓ All context manager tests passed!")

**Context Manager Patterns:**

1. **Class-based:**
   - More explicit and flexible
   - Implement `__enter__` and `__exit__`
   - Good for complex state management

2. **Generator-based:**
   - More concise
   - Use `@contextmanager` decorator
   - Good for simple cases

**Common Use Cases:**
- Resource management (files, connections)
- Lock acquisition/release
- Temporary state changes
- Setup/teardown operations

**Follow-up Questions:**
- How to implement nested transactions?
- How to add timeout handling?
- What about async context managers?

### Problem 5: Timer Context Manager (Easy)

**Problem Statement:**
Create a context manager that measures execution time of a code block.

In [ ]:
import time
from contextlib import contextmanager

class Timer:
    """Context manager to measure execution time"""
    
    def __init__(self, name: str = "Block"):
        self.name = name
        self.elapsed = 0
    
    def __enter__(self):
        self.start = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.elapsed = time.time() - self.start
        print(f"{self.name} took {self.elapsed:.4f} seconds")
        return False

@contextmanager
def timer(name: str = "Block"):
    """Generator-based timer context manager"""
    start = time.time()
    try:
        yield
    finally:
        elapsed = time.time() - start
        print(f"{name} took {elapsed:.4f} seconds")

# Test the timer
with Timer("Sleep test") as t:
    time.sleep(0.1)
assert t.elapsed >= 0.1

with timer("Another test"):
    sum(range(100000))

print("✓ Timer context manager tests passed!")

<a id='generators'></a>
## 3. Generators & Iterators

### Problem 6: Implement itertools.groupby (Medium)

**Problem Statement:**
Implement a generator function similar to `itertools.groupby` that groups consecutive elements by a key function.

**Examples:**
```python
data = [1, 1, 2, 2, 2, 3, 1, 1]
for key, group in groupby(data):
    print(key, list(group))
# Output:
# 1 [1, 1]
# 2 [2, 2, 2]
# 3 [3]
# 1 [1, 1]
```

In [ ]:
from typing import Iterable, Callable, Any, Iterator, Tuple

def groupby(iterable: Iterable, key: Callable = None) -> Iterator[Tuple[Any, Iterator]]:
    """
    Group consecutive elements with the same key.
    
    Args:
        iterable: Input sequence
        key: Function to compute key (identity if None)
    
    Yields:
        (key, group_iterator) tuples
    """
    if key is None:
        key = lambda x: x
    
    iterator = iter(iterable)
    
    # Get first element
    try:
        current = next(iterator)
    except StopIteration:
        return
    
    current_key = key(current)
    group = [current]
    
    for item in iterator:
        item_key = key(item)
        
        if item_key == current_key:
            # Same group
            group.append(item)
        else:
            # New group - yield previous group
            yield current_key, iter(group)
            current_key = item_key
            group = [item]
    
    # Yield last group
    yield current_key, iter(group)

# Test groupby
data = [1, 1, 2, 2, 2, 3, 1, 1]
result = [(k, list(g)) for k, g in groupby(data)]
assert result == [(1, [1, 1]), (2, [2, 2, 2]), (3, [3]), (1, [1, 1])]

# Test with key function
words = ['apple', 'apricot', 'banana', 'blueberry', 'cherry']
result = [(k, list(g)) for k, g in groupby(words, key=lambda x: x[0])]
assert result[0][0] == 'a'
assert len(result[0][1]) == 2

print("✓ All groupby tests passed!")

**Generator Best Practices:**
- Use generators for large or infinite sequences
- Memory efficient - lazy evaluation
- Can only iterate once
- Use `yield` to produce values

**Generator Expressions vs List Comprehensions:**
```python
# List comprehension - eager, uses memory
squares = [x**2 for x in range(1000000)]

# Generator expression - lazy, memory efficient
squares = (x**2 for x in range(1000000))
```

### Problem 7: Implement a Custom Range Iterator (Medium)

**Problem Statement:**
Create a custom iterator class that behaves like Python's `range()` but supports float steps.

In [ ]:
class FloatRange:
    """
    Iterator that supports floating-point steps.
    Implements iterator protocol (__iter__ and __next__).
    """
    
    def __init__(self, start, stop=None, step=1.0):
        if stop is None:
            # Single argument: range(n)
            self.start = 0.0
            self.stop = float(start)
        else:
            # Two or three arguments: range(start, stop, step)
            self.start = float(start)
            self.stop = float(stop)
        
        self.step = float(step)
        
        if self.step == 0:
            raise ValueError("step cannot be zero")
        
        self.current = self.start
    
    def __iter__(self):
        """Return the iterator object (self)"""
        return self
    
    def __next__(self):
        """Return the next value"""
        # Check if we've reached the end
        if self.step > 0:
            if self.current >= self.stop:
                raise StopIteration
        else:
            if self.current <= self.stop:
                raise StopIteration
        
        value = self.current
        self.current += self.step
        return value
    
    def __repr__(self):
        return f"FloatRange({self.start}, {self.stop}, {self.step})"

# Test FloatRange
result = list(FloatRange(0, 1, 0.2))
assert len(result) == 5
assert abs(result[0] - 0.0) < 0.001
assert abs(result[-1] - 0.8) < 0.001

# Test negative step
result = list(FloatRange(1, 0, -0.25))
assert len(result) == 4
assert abs(result[0] - 1.0) < 0.001

print("✓ All FloatRange tests passed!")

**Iterator Protocol:**
1. `__iter__()`: Returns iterator object (usually `self`)
2. `__next__()`: Returns next value or raises `StopIteration`

**Iterator vs Iterable:**
- **Iterable:** Object with `__iter__()` that returns an iterator
- **Iterator:** Object with `__iter__()` and `__next__()`
- All iterators are iterables, but not all iterables are iterators

**Follow-up Questions:**
- How to make it support `len()`? (Implement `__len__`)
- How to make it reusable? (Reset in `__iter__`)
- What about infinite iterators?

<a id='metaclasses'></a>
## 4. Metaclasses

### Problem 8: Singleton Metaclass (Hard)

**Problem Statement:**
Implement a metaclass that ensures only one instance of a class can exist (Singleton pattern).

**Requirements:**
- Thread-safe
- Works with any class
- Preserves class functionality

In [ ]:
import threading
from typing import Any, Dict

class SingletonMeta(type):
    """
    Metaclass that creates Singleton classes.
    Thread-safe implementation using locks.
    """
    
    _instances: Dict[type, Any] = {}
    _lock: threading.Lock = threading.Lock()
    
    def __call__(cls, *args, **kwargs):
        """
        Called when creating an instance of the class.
        Overrides default instance creation.
        """
        # Double-checked locking for thread safety
        if cls not in cls._instances:
            with cls._lock:
                # Check again inside lock
                if cls not in cls._instances:
                    instance = super().__call__(*args, **kwargs)
                    cls._instances[cls] = instance
        
        return cls._instances[cls]

# Usage example
class Database(metaclass=SingletonMeta):
    """Database connection singleton"""
    
    def __init__(self, connection_string: str = "default"):
        self.connection_string = connection_string
        self.connected = True

# Test singleton behavior
db1 = Database("postgres://localhost")
db2 = Database("mysql://localhost")  # Same instance!

assert db1 is db2
assert db1.connection_string == "postgres://localhost"
assert db2.connection_string == "postgres://localhost"  # Not changed

print("✓ Singleton metaclass test passed!")

**Metaclass Fundamentals:**
- Metaclass is a class of a class
- `type` is the default metaclass
- Metaclasses control class creation
- Override `__new__` or `__call__` for custom behavior

**When to Use Metaclasses:**
- Enforcing coding standards (validation)
- Registering classes automatically
- Singleton pattern
- ORM frameworks (like Django models)
- API frameworks

**Alternatives to Metaclasses:**
- Decorators (simpler, more common)
- `__init_subclass__` (Python 3.6+)
- Module-level instances

**Quote:**
> "Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don't." - Tim Peters

### Problem 9: Validation Metaclass (Hard)

**Problem Statement:**
Create a metaclass that automatically validates class attributes based on type annotations.

In [ ]:
from typing import get_type_hints

class ValidatedMeta(type):
    """
    Metaclass that adds type validation to __init__.
    Uses type annotations to validate arguments.
    """
    
    def __new__(mcs, name, bases, namespace, **kwargs):
        # Get the new class
        cls = super().__new__(mcs, name, bases, namespace)
        
        # Get type hints for the class
        hints = get_type_hints(cls)
        
        # Store original __init__
        original_init = cls.__init__
        
        def validated_init(self, **kwargs):
            """Wrapper that validates types before calling original __init__"""
            for name, value in kwargs.items():
                if name in hints:
                    expected_type = hints[name]
                    if not isinstance(value, expected_type):
                        raise TypeError(
                            f"{name} must be {expected_type.__name__}, "
                            f"got {type(value).__name__}"
                        )
            
            # Set validated attributes
            for name, value in kwargs.items():
                setattr(self, name, value)
        
        cls.__init__ = validated_init
        return cls

# Usage example
class Person(metaclass=ValidatedMeta):
    name: str
    age: int
    
    def __repr__(self):
        return f"Person(name={self.name}, age={self.age})"

# Test validation
person = Person(name="Alice", age=30)
assert person.name == "Alice"
assert person.age == 30

# Test type error
try:
    Person(name="Bob", age="thirty")  # age should be int
    assert False, "Should have raised TypeError"
except TypeError as e:
    print(f"✓ Validation working: {e}")

print("✓ All validation metaclass tests passed!")

## Summary

### Python Features Mastery

1. **Decorators:**
   - Function wrappers
   - Parameterized decorators
   - Class decorators
   - `functools.wraps`

2. **Context Managers:**
   - `__enter__` and `__exit__`
   - `@contextmanager` decorator
   - Resource management
   - Exception handling

3. **Generators:**
   - `yield` keyword
   - Memory efficiency
   - Iterator protocol
   - Generator expressions

4. **Metaclasses:**
   - Class creation control
   - `__new__` and `__call__`
   - Type validation
   - Singleton pattern

### Interview Tips

- **Explain the "why"**: Don't just implement, explain when and why to use each feature
- **Trade-offs**: Discuss alternatives and their pros/cons
- **Performance**: Understand memory and time implications
- **Best practices**: Know when NOT to use advanced features
- **Real-world usage**: Give practical examples

### Common Pitfalls

1. Forgetting `functools.wraps` in decorators
2. Not handling exceptions in context managers
3. Confusing iterators and iterables
4. Overusing metaclasses (decorators are often better)
5. Not considering thread safety

### Additional Topics to Study

- `@property` and descriptors
- `__getattr__`, `__setattr__`, `__getattribute__`
- `__new__` vs `__init__`
- Abstract base classes (ABC)
- Type hints and `typing` module
- Async/await and coroutines
- Python data model (special methods)